# JumpStart로 시작하는 Llama 3.2 1B 배포 (Deployment)
- JumpStart에 Prebuild 되어 있는 Llama 3.2 1B Instruction 모델을 가져 옵니다.
- 이 때, EULA에 동의하는 `accept_eula=True` 설정을 해 주어야 합니다. 

In [1]:
import warnings
warnings.filterwarnings(action='ignore') # 경고 메시지를 무시

In [2]:
from sagemaker.jumpstart.model import JumpStartModel

model_id, model_version = "meta-textgeneration-llama-3-2-1b-instruct", "*"
accept_eula = True
instance_type='ml.g6.xlarge' # Default 
endpoint_name='basic-jumpstart-deploy-llama3-2-1b-01' # 원하는 명칭으로... ('_' 허용하지 않음)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


위에서는 다음에 사용될 설정들을 미리 변수로 지정 하였습니다. 

이제 JumpStartModel을 로딩합니다.
- model_id와 버전을 위 변수에 세팅한것과 같이 설정 해 줍니다.
- 인스턴스는 `instance_type="ml.g6.xlarge"`이 기본 값입니다. 설정을 안해도 됩니다.

In [3]:
model = JumpStartModel(model_id=model_id, model_version=model_version, instance_type=instance_type)

Using model 'meta-textgeneration-llama-3-2-1b-instruct' with wildcard version identifier '*'. You can pin to version '1.2.1' for more stable results. Note that models may have different input/output signatures after a major version upgrade.


가장 단순하게는 `predictor = model.deploy()`만으로 실행 가능합니다. (기본설정 사용)
- 그러나, llama 3.2 경우, EULA 동의를 필요로 하기 때문에, `predictor = model.deploy(accept_eula=accept_eula)`가 최소 사항
- 추가적으로 `endpoint_name` 정도 추가 지정이 가능합니다.

In [4]:
predictor = model.deploy(accept_eula=accept_eula, endpoint_name=endpoint_name)

------------!

# 추론 (Inference) 
이제 Llama 3.2 1B 모델이 배포 되었습니다. 이제 추론을 테스트 해 볼 예정입니다. 
- 추론은 크게 2가지 방법으로 테스트 할 수 있습니다. 
- 1/ predictor에 직접 호출 하는 방법
- 2/ 실제 외부에서 network로 API 호출하는 방법



## 1. 데이터 포맷
Llama 3.2는 훈련/추론 할 때 **특정 데이터 포맷** 형태로 이루어져 있고, 이 포맷을 엄격하게 따라야 합니다. 대략적인 데이터 포맷은, prompt 문장에서 `각 세그먼트를 구분하는 스페셜 토큰들`이 포함되어 있다는 점 입니다. 
다음은 사용되는 스페셜 토큰들 입니다. 
- `<|begin_of_text|>` : 프름프트의 시작
- `<|start_header_id|>` , `<|end_header_id|>` : Role을 지정할 때 사용. ex) `<|start_header_id|>system<|end_header_id|>`
- `<|eot_id|>` : 하나의 역할이 종료되었음을 알림. 종료 메세지.

### 템플릿
이러한 데이터 포맷에 따르도록 템플릿을 생성하고, 해당 템플릿에 instruction일 채워 넣음
```python
<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n
{instruction}<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>\n\n
```
LLM의 특성 상, 위 문자열은 자연스럽게 다음 문장을 작성하기 위해, Assistant의 Response를 요구함.


In [5]:
llama_template_train = """
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a helpful assistant<|eot_id|><|start_header_id|>user<|end_header_id|>
{instruction}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
{response}<|eot_id|>
"""
llama_template_inference = """
<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n{instruction}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n
"""

## 2. config
궁극적으로 지금의 추론 단계는 `predictor.predict()`을 수행하는 것을 목적으로 한다. 이 때 predict()함수는 전송형태 (dict)의 페이로드를 요구한다. 
- payload는 `{ "inputs" : "프롬프트 포맷", "parameters": {추론 파라미터들} }` 형태에 따른다.
- 가장 단순한 예는 다음과 같다. (여기서 prompt는 프롬프트 포맷 형태의 문자열이다.)
```
prompt_payload = {
    'inputs':prompt,
    'parameters': {'max_new_tokens': 256, 'top_p': 0.9,'temperature': 0.6, 'details': True}
}
response = predictor.predict(prompt_payload)

In [6]:
prompt = llama_template_inference.format(instruction="AWS Sagemaker에 대해 소개해 주세요. ")
prompt_payload = {
    'inputs':prompt,
    'parameters': {'max_new_tokens': 256, 'top_p': 0.9,'temperature': 0.6, 'details': True}
}

In [7]:
response = predictor.predict(prompt_payload)

In [8]:
response

{'generated_text': 'AWS Sagemaker는 AWS의 cloud-based machine learning (ML)aaS (Application Service for Machine Learning) 플랫폼입니다. AWS Sagemaker는 MLaaS를 제공하는 AWS의 cloud-based ML 플랫폼 중 하나입니다. AWS Sagemaker는 MLaaS를 사용하여 ML 모델을 배포, 배포, 유지, 및 확장할 수 있는 플랫폼입니다. \n\nAWS Sagemaker는 ML 모델을 배포, 배포, 유지, 및 확장할 수 있는 플랫폼으로, ML 모델을 배포, 배포, 유지, 및 확장할 수 있는 플랫폼입니다. AWS Sagemaker는 ML 모델을 배포, 배포, 유지, 및 확장할 수 있는 플랫폼으로, ML 모델을 배포, 배포, 유지, 및 확장할 수 있는 플랫폼입니다.\n\nAWS Sagemaker는 MLaaS를 사용하여 ML 모델을 배포, 배포, 유지, 및 확장할 수 있는 플랫폼으로, ML 모델을 배포, 배포, 유지, 및 확장할 수 있는 플랫폼으로, ML 모델을 배포, 배포, 유지, 및 확',
 'details': {'finish_reason': 'length',
  'generated_tokens': 256,
  'inputs': '\n<|begin_of_text|><|start_header_id|>user<|end_header_id|>\nAWS Sagemaker에 대해 소개해 주세요. <|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n\n',
  'tokens': [{'id': 37236, 'text': 'AWS', 'log_prob': -0.024860398843884468},
   {'id': 328, 'text': ' S', 'log_prob': -0.2083704173564911},
   {'id': 15003, 'text': 'agem', 'log_prob': -0.0054842964746

In [9]:
print(response.keys())
print()
print(response['generated_text'])

dict_keys(['generated_text', 'details'])

AWS Sagemaker는 AWS의 cloud-based machine learning (ML)aaS (Application Service for Machine Learning) 플랫폼입니다. AWS Sagemaker는 MLaaS를 제공하는 AWS의 cloud-based ML 플랫폼 중 하나입니다. AWS Sagemaker는 MLaaS를 사용하여 ML 모델을 배포, 배포, 유지, 및 확장할 수 있는 플랫폼입니다. 

AWS Sagemaker는 ML 모델을 배포, 배포, 유지, 및 확장할 수 있는 플랫폼으로, ML 모델을 배포, 배포, 유지, 및 확장할 수 있는 플랫폼입니다. AWS Sagemaker는 ML 모델을 배포, 배포, 유지, 및 확장할 수 있는 플랫폼으로, ML 모델을 배포, 배포, 유지, 및 확장할 수 있는 플랫폼입니다.

AWS Sagemaker는 MLaaS를 사용하여 ML 모델을 배포, 배포, 유지, 및 확장할 수 있는 플랫폼으로, ML 모델을 배포, 배포, 유지, 및 확장할 수 있는 플랫폼으로, ML 모델을 배포, 배포, 유지, 및 확


response의 결과를 보면, 2개의 카값을 가진다. `generated_text`, `details`
- `generated_text`: 우리가 주로 원하던 정답이다. 
- `details`: 우리가 추론 파라미터에 `'details': True` 설정값을 주었기 때문에 생성된 자세한 내용이다.
  - Token ID, Token Text, Log_prob를 상세하게 볼 수 있다. --> **아~ 토큰이 이렇게 나누어 져 있구나, 확률값은 이정도 값이구나.. 하는 공부가 될것이다.**
  - 모든 토큰에 대한 log_prob 중에서 해당 토큰이 선택 되었고 (선택 전략에 따라 가장 높은값), 이 토큰의 확률의 로그값이다.
  - Ground Truth와는 다른 log_prob는 Distillation에도 사용이 된다.

## 응답값 반복 문제

그런데, `모델 응답`의 값을 살펴 보면 조금 이상하다. `... 모델을 배포, 배포, 유지, 및 확장할수 있는 플랫폼으로 ML 모델을 배포, 배포, 유지, 및 ....`
> 응? 뭐지? 이상한데?

뭔가 모르게 단어도 그렇고, 문장도 그렇게 계속 반복 되는 것 같다. 동일한 문장 전체가 반복되기도 한다. 
이러한 현상은, Lllama 등 오픈소스 모델에서 자주 발생하는 현상으로, Llama는 이러한 현상을 방지하기 위한 설정값들을 제공하고 있다. 
- 문장의 반복 : `eos_token 설정` - 말이 끝났으면 더이상 쓸데없는 말 만들지 말고 종료해라!
- 반복 페널티 : `repetition_penalty` - 트랜스포머의 특징상 다음단어를 선택할 때 동일하거나 유사한 단어가 선택될 확률이 높은 데, 동일한 선택이 있을 경우 확률을 낮추는 페널티텀을 추가하고 이것의 비율을 지정한다. `1.0`이 기준이고 1.0보다 크면 페널티를 준다. 보통 `1.2` 사용
- do_sample : 마찬가지로 다음단어가 동일하게 선택하지 말고, 확률 기반으로 재샘플링 하는 방식을 실행 한다. (어라. 이거 particle filter에서 사용하던 알고리즘.) 당연하게도 변수를 만드는 부분이므로 temperature와 연관이 높다.


In [10]:
# 다시 파라미터를 재정비 한다. # EOS Token이 없으면, 문장을 끝내지 않고 반복함.
# 파라미터 정비 후 payload 재작성
generation_config = {
    "max_new_tokens": 256,
    "eos_token_id": 128009,          # Llama 3.2 공식 EOS 토큰[1][3]
    "pad_token_id": 128009,          # 패딩 토큰 통일[4]
    "repetition_penalty": 1.2,       # 반복 문구 억제[3]
    "temperature": 0.6,    
    "do_sample": True,
    "top_p": 0.9,
    'details': True,
}

prompt_payload = {
    'inputs':prompt,
    'parameters': generation_config
}

In [11]:
response = predictor.predict(prompt_payload)

In [12]:
print(response.keys())
print("---")
print(response['generated_text'])


dict_keys(['generated_text', 'details'])
---
AWS Sagemaker는 AWS의 AI 및 machine learning을 위한 고성능 인프라를 제공하는 cloud-based platform입니다.它은 대규모-scale machine learning (ML)aaS(아이티드 서비스)와 클라우dscape에서 ML model을 배포, Runs 및 manages 하도록 도움을 줄 수 있습니다.

Sagemaker는 다음과 같은feature가 있습니다.
- **대규모-scale ML** : AWS Sagemaker에서는 대규모-scale ML 환경을 구축할 때부터 시작하여 다양한 ML 기법을 사용하여 모델을 배포하고 Run을 관리할 수 있습니다. 

- **클라우dTuner**: AWS Sagemaker에는 클라우dTuner API를 통해 CLT (Cloud Tuner)는 CLI (Command Line Interface)를 이용하여 ML Model을 Tuning 하기위한 최적화 전략과 시트를 생성합니다. 

- **Model Store**: AWS Sagemaker Model Store는 AWS CloudWatch Data Lake Hadoop에モデル lưu trữ할 수 있는 확장된 storage service입니다. 

- **Containerization**: AWS Sagemaker provides pre-built container images for popular frameworks such as TensorFlow and PyTorch. 

- **Environment Variables**: AWS Sagem
